In [35]:
import pandas as pd
import numpy as np
import nltk 
import re

from collections import Counter

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster.hierarchical import AgglomerativeClustering

In [36]:
emails = pd.read_csv("emails/Emails.csv")
persons = pd.read_csv("emails/Persons.csv")

In [37]:
emails.head()

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
0,1,C05739545,WOW,H,"Sullivan, Jacob J",87.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...,F-2015-04841,...,NaN,"Sullivan, Jacob J <Sullivan11@state.gov>",NaN,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,05/13/2015,RELEASE IN FULL,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
1,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,NaN,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...
2,3,C05739547,CHRIS STEVENS,;H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739547...,F-2015-04841,...,B6,"Mills, Cheryl D <MillsCD@state.gov>","Abedin, Huma","Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,05/14/2015,RELEASE IN PART,Thx,UNCLASSIFIED\nU.S. Department of State\nCase N...
3,4,C05739550,CAIRO CONDEMNATION - FINAL,H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739550...,F-2015-04841,...,NaN,"Mills, Cheryl D <MillsCD@state.gov>","Mitchell, Andrew B","Wednesday, September 12,2012 12:44 PM",F-2015-04841,C05739550,05/13/2015,RELEASE IN PART,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
4,5,C05739554,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,"Abedin, Huma",H,80.0,2011-03-11T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739554...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739554,05/13/2015,RELEASE IN PART,"H <hrod17@clintonemail.com>\nFriday, March 11,...",B6\nUNCLASSIFIED\nU.S. Department of State\nCa...


In [38]:
persons.head()

,Id,Name
0,1,111th Congress
1,2,AGNA USEMB Kabul Afghanistan
2,3,AP
3,4,ASUNCION
4,5,Alec


## Изучение датасета
Сразу бросается в глаза количество колонок с метаданными, которые нам не очень интересны. Также, поле RawText, пусть и содержит тексты писем, является менее полезным, чем ExtractedBodyText -- его не нужно так тщательно очищать. Тема письма, вероятно, тоже поможет при кластеризации.

Поэтому будем работать с тремя полями: Persons.Name, Emails.ExtractedBodyText и Emails.ExtractedSubject

In [39]:
data = pd.merge(emails, persons, left_on="SenderPersonId", right_on="Id")
data = data[["ExtractedSubject", "ExtractedBodyText", "Name"]]
data = data.rename(columns={"ExtractedSubject": "Subject", "ExtractedBodyText": "Text"})
data = data.dropna(how='any', subset=["Text", "Subject"])
data.head()

,Subject,Text,Name
1,FVV: Secretary's remarks,FYI,Jake Sullivan
2,AbZ and Hb3 on Libya and West Bank/Gaza,Fyi\nB6\n— —,Jake Sullivan
3,hey,Fyi,Jake Sullivan
4,Fw: S today,This is nice.,Jake Sullivan
6,Re: Proposed Quad Deal,B5,Jake Sullivan


In [40]:
plaintext = zip(data["Subject"].tolist(), data["Text"].tolist())

In [41]:
to_delete = ["b6", "b5", "--", "re:", "h:", "case no.", "doc no.", "date:", "state dept", "prom:", "sent", 
             "to:", "cc:", "fw:", "fvv:", "from:", "for:", "tel:", "fax:", "b1", "b2", "b3", "b4", "1.4(", 
             "#1", "1#", "release in", "u.s. department of state", "subject to agreement", 
             "ent of state", "state-", "attachment=", "january", "february", "march", "april", "may", 
             "june", "july", "august", "september", "october", "december", "http"]

new_text = []
for text in plaintext:
    key = text[0].lower()
    for i in to_delete:
        while i in key:
            key = key.replace(i, "")
            
    if key:
        new_text.append([key])
    else:
        new_text.append([])
    
    for row in text[1].split("\n"):
        new_line = row.lower()
        for i in to_delete:
            if i in new_line:
                new_line = ""
                break
        
        if new_line:
            new_text[-1].append(new_line)
    
    new_text[-1] = " ".join(new_text[-1])

new_text[15:20]

['speech draft for friday at csis sorry we were a couple hours late!',
 " speech draft for friday at csis survive, yes. pat helped level set things tonight and we'll see where we are in the morning.",
 ' hrc @ csis - v8 here you go',
 ' q re hbj fyi',
 '  ubl, aq & libya. sid strikes me as a little strange. but certainly disturbing. i will pass info on. from']

In [42]:
good_pattern = re.compile("[a-z0-9]")
bad_words = stopwords.words("english") + ["fyi"]

good_text = []
for line in new_text:
    tokens = word_tokenize(line)
    good_tokens = [i for i in tokens if i not in bad_words and good_pattern.match(i) and i.isalnum()]
    
    if good_tokens:
        good_text.append(good_tokens)
    
good_text[15:20]

[['speech', 'draft', 'friday', 'csis', 'sorry', 'couple', 'hours', 'late'],
 ['speech',
  'draft',
  'friday',
  'csis',
  'survive',
  'yes',
  'pat',
  'helped',
  'level',
  'set',
  'things',
  'tonight',
  'see',
  'morning'],
 ['hrc', 'csis', 'v8', 'go'],
 ['q', 'hbj'],
 ['ubl',
  'aq',
  'libya',
  'sid',
  'strikes',
  'little',
  'strange',
  'certainly',
  'disturbing',
  'pass',
  'info']]

In [43]:
ngrams = Counter()
for tokens in good_text:
    for pair in [(tokens[i], tokens[i + 1]) for i in range(len(tokens) - 1)]:
            ngrams[str(pair[0]) + " " + str(pair[1])] += 1

for pair in ngrams.most_common(5):
    print(str(pair))

('secretary office', 367)
('state department', 334)
('united states', 318)
('white house', 276)
('pm secretary', 216)


Кажется, топ-5 биграмм не очень информативны, поскольку в них нет новой для нас информации: мы и так знаем, что Хиллари Клинтон была US Secretary of State и сидела в Белом Доме

In [44]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
all_words = []
for line in good_text:
    for token in line:
        all_words.append(token)
finder = BigramCollocationFinder.from_words(all_words)
finder.apply_freq_filter(3)
finder.nbest(bigram_measures.pmi, 10)

[('630pm', 'shaw'),
 ('buenos', 'aires'),
 ('dalai', 'lama'),
 ('irwin', 'redlener'),
 ('jassim', 'jabr'),
 ('kin', 'notifications'),
 ('kuan', 'yew'),
 ('noor', 'ramsey'),
 ('phd', 'purdue'),
 ('tick', 'tock')]

Коллокации уже повеселее, они указывают на места и на конкретных личностей.
 * 630 (at) shaw (campus) -- видимо, относит к выступлению перед выпускниками в школе Shaw.
 * Буэнос-Айрес, Далай Лама -- ясно-понятно.
 * Ирвин Рэдленер -- педиатр и спикер, основатель Children's Health Fund.
 * Хамад бен Джасим бен Джабер Аль Тани -- бывший премьер-министр Катара.
 * (Next of) kin notification -- "похоронка", божечки. Упоминалась в контексте скандала в Бенгази, после убийства 4 американских граждан. Ходили слухи, что миссис Клинтон знала о предстоящей атаке, но не предупредила.
 * Ли Куан Ю -- тот самый президент Сингапура.
 * Нур и Рамси -- двое похищенных приёмной матерью детей, которых отвезли в Египет, и, кажется, вернули отцу. Кажется, это было большим скандалом в 2009.
 * PhD Purdue -- степень доктора наук в Университете Пердью.
 * (Libya) Tick Tock -- самая интересная коллокация. Письмо, которое якобы доказывает причастность Хиллари к перевороту и войне в Ливии, хронология всех её участий в заседаниях НАТО. Легко гуглится, сложно пересказывается.
 
Такая простая обработка, такие любопытные результаты! Но перейдём к кластеризации.